In [1]:
import os
import numpy as np
#import statistics as stats
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import feature_selection, linear_model , metrics,discriminant_analysis , preprocessing , neighbors , cross_validation
from sklearn import utils
pd.set_option('display.max_rows', 15)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 15)

%matplotlib inline
plt.style.use('ggplot')

In [2]:
df = pd.read_csv("Boston.csv")

In [3]:
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [6]:
#Calculating the population Mean of medv
mu_hat = df.medv.mean()
print "The estimated population mean is --- %r" %mu_hat

The estimated population mean is --- 22.532806324110698


In [7]:
df.medv.describe()

count    506.000000
mean      22.532806
std        9.197104
min        5.000000
25%       17.025000
50%       21.200000
75%       25.000000
max       50.000000
Name: medv, dtype: float64

As evident , the std dev for medv is 9.18
std err =  sample mean/sqroot of n

In [13]:
df.medv.std()

9.19710408737982

In [14]:
root_n = np.sqrt(len(df))
std_err = df.medv.std()/root_n
print std_err

0.408861147498


In [17]:
#creating an empty DataFrame to hold the values for the Bootstrappng 
df_boot = pd.DataFrame({'iteration':np.NAN , 'mu_hat':np.NAN} , index = [0])

In [18]:

df_boot.head()

,iteration,mu_hat
0,NaN,NaN


In [47]:
# This loop calculates the bootstrapping, runs 500 times and randonly selects the sample from the Data set with replacement 
for i in range(0 , 500):
    medv_hat = df.medv.sample(frac =1  , replace = True )
    mu_medv_hat = medv_hat.mean()
    #print mu_medv_hat
    df_boot.ix[i ,'iteration'] = i+1
    df_boot.ix[i ,'mu_hat'] = mu_medv_hat
    

In [28]:
df_boot.head()

,iteration,mu_hat
0,1.0,22.424506
1,2.0,22.234190
2,3.0,21.841304
3,4.0,22.673715
4,5.0,22.584783


In [29]:
boot_strapped_mu = df_boot.mu_hat.mean()

In [30]:
# a new feature to hold the std dev
df_boot['boot_err'] = (df_boot.mu_hat - boot_strapped_mu)**2

In [32]:
df_boot.head()

,iteration,mu_hat,boot_err
0,1.0,22.424506,0.010603
1,2.0,22.234190,0.086018
2,3.0,21.841304,0.470835
3,4.0,22.673715,0.021385
4,5.0,22.584783,0.003284


In [37]:
boot_strapped_err = ((df_boot.boot_err.sum())/(len((df_boot) -1)))
boot_strapped_err = np.sqrt(boot_strapped_err)

print " Bot strapped error is --%r" %boot_strapped_err

 Bot strapped error is --0.39200671684287608


In [43]:
# COmputing 95% confidence interval on mu_hat
lower_bound = boot_strapped_mu - 2*boot_strapped_err
upper_bound = boot_strapped_mu + 2*boot_strapped_err

print lower_bound
print upper_bound

21.7434652224
23.3114920898


# Median bootstrapped

In [44]:
df_boot['Median'] = np.NAN

In [48]:
for i in range(0 , 500):
    sample = df.medv.sample(frac =1  , replace = True )
    medv_median_hat = sample.median()
    #print mu_medv_hat
    #df_boot.ix[i ,'iteration'] = i+1
    df_boot.ix[i ,'Median'] = medv_median_hat
    

In [49]:
df_boot.head()

,iteration,mu_hat,boot_err,Median
0,1.0,22.819763,0.010603,21.7
1,2.0,22.094466,0.086018,21.4
2,3.0,22.838538,0.470835,20.6
3,4.0,22.922134,0.021385,20.6
4,5.0,22.465810,0.003284,21.4


In [50]:
df_boot.Median.describe()

count    500.000000
mean      21.181900
std        0.373022
min       20.000000
25%       20.900000
50%       21.200000
75%       21.412500
max       22.150000
Name: Median, dtype: float64

In [54]:
std_err_median = df_boot.Median.std() / (len(df_boot)-1)
print std_err_median

0.000747539800412


0.373022360406
